# Marco Learning Corrections

Marco learning is a separate file of results that is beyond the Illuminate reports that are used for the other Jupyter notebooks. In this notebook, the raw data file that is received from Marco learning is manipulated and combined with the Illuminate files to generate a new file for the overall performance of students on the test as well as the standard that they performed on.

In [ ]:
import numpy as np
import pandas as pd

## Input Destination

You need to input the destination folder and give it a file name at the end before running the full code.

In [ ]:
# Write pathway and file name for overall file

overall_final = output_final = "C:\\Users\\derek.castleman\\Desktop\\CFA Data\\CFA Data 23-24\\CFA 2\\CFA_Overall_Marco.xlsx"

standards_final = output_final = "C:\\Users\\derek.castleman\\Desktop\\CFA Data\\CFA Data 23-24\\CFA 2\\CFA_Standards_Marco.xlsx"

## Uploading Files

The input for the marco learning raw file will be asked for. Since some students have a blank for a score, these will be filled in with a 0. The file pathway for the overall Illuminate scores and the Standards report will also be requested for an input.

In [ ]:
# The pathway for the Marco learning file will be asked to input
marco = input('Pathway of raw Marco learning data (get rid of quotations):   ')
marco = pd.read_excel(marco)

In [ ]:
marco

In [ ]:
# The pathway for Overall Illuminate scores will be asked for
overall = input('Pathway of raw Illuminate learning data (get rid of quotations):   ')
overall = pd.read_excel(overall)

In [ ]:
overall

In [ ]:
# The pathway for the overall standards will be asked for
overall_standards = input('Pathway of raw Illuminate Standards learning data (get rid of quotations):   ')
overall_standards = pd.read_excel(overall_standards)

In [ ]:
# The Marco Learning dataframe will be reduced to the two columns of interest
marco = marco[['student_id', 'Total']]
marco

In [ ]:
# The blank scores will be filled in with a 0
marco = marco.fillna(0)
marco

## Fixing Overall Scores

The Marco learning will be combined with the overall scores dataframe. A left merge will be done so that students who did not complete a Marco learning test. Then the column of the total will be used to fill in for the incorrect score that is reflected in Illuminate.

The percentage correct will be calculated and turned into a string with a % sign since this is the standard format that Illuminate puts the overall percentage as.

In [ ]:
# Left merge is done with the overall and the Marco learning file
combined = pd.merge(overall, marco, how='left', left_on='Student Id', right_on='student_id')
combined

In [ ]:
# Any duplicates are dropped
combined = combined.drop_duplicates(subset=['Student Id'])
combined

In [ ]:
# Number correct in the overall is replaced with the total from Marco learning if there is a value there
combined['Number Correct'] = combined.apply(lambda row: row['Total']
                            if pd.notna(row['Total']) else row['Number Correct'], axis=1)
combined

In [ ]:
# Performance band level is also replaced with the Marco learning total if there is a value there
combined['Performance Band Level'] = combined.apply(lambda row: row['Total']
                            if pd.notna(row['Total']) else row['Performance Band Level'], axis=1)
combined

In [ ]:
# The percentage is calcualted with the new Number Correct and changed into a string with a % sign
combined['Percentage'] = ((combined['Number Correct'] / combined['Number Possible']) * 100).round(2).astype(str) + '%'
combined

In [ ]:
# The % Correct column is set equal to percentage
combined['% Correct'] = combined['Percentage']
combined

In [ ]:
# Extra columns are dropped to recreate the overall sheet fixed without Marco Learning
columns_to_drop = ['student_id', 'Total', 'Percentage']
final_overall = combined.drop(columns=columns_to_drop)
final_overall

## Fixing the Standards

The overall corrected with the Marco Learning dataframe still attached will be used. Percentage will be calculated again but instead of being a string it will be a number this time since this is how it is in the Illuminate file.

The Performance Band Level and the % Correct will be selected. And then a left merge will be done so that students who did not take Marco learning are not eliminated.

The dataframe is melted with a column for standard and another with its value.

Two dataframes are created: marco learning and non-marco learning.

Where the is a column in the performance band standard column, the level value will be replaced with the performance band value. Where there is a %Correct in the standard column, the level will be replaced with the value for the percent correct.

The rows without performance band for standard and % correct are selected then concatenated with the dataframes that were just fixed for level and % correct. This is then concatenated with the non-marco students. Then the data is unstacked to create a file that is for the standards with the Marco tests now corrected in it.

In [ ]:
overall_standards

In [ ]:
combined

In [ ]:
# Dropping the percentage column
columns_to_drop = ['Percentage']
combined = combined.drop(columns=columns_to_drop)

In [ ]:
combined

In [ ]:
# Recalculating percent but as a number instead of a string this time
combined['Percentage'] = ((combined['Number Correct'] / combined['Number Possible']) * 100).round(2)
combined

In [ ]:
# Changing the % Correct with the newly calculated percentage number
combined['% Correct'] = combined['Percentage']
combined

In [ ]:
# Selecting the three columns of interest from the combined dataframe
marco_standards = combined[['student_id', 'Performance Band Level', '% Correct']]
marco_standards

In [ ]:
overall_standards

In [ ]:
# A left merge is done with the overall standards and the marco standards
combined_standards = pd.merge(overall_standards, marco_standards, how='left', left_on='Student ID', right_on='student_id')
combined_standards

In [ ]:
# Data is melted with the standards columns being one and the value being a level
melted = combined_standards.melt(['Current Site ID','Student ID', 'Current Grade Level', 'State Student ID',
             'First Name', 'Last Name', 'English Proficiency Code',
            'English Proficiency', 'Student Is Special Ed?', 'Date Taken', 'student_id',
            'Performance Band Level_y', '% Correct'], var_name='Standard', value_name='Level')
melted

In [ ]:
# Creating a dataframe with non-marco students by selecting those who do not have a marco id
non_marco = melted[melted['student_id'].isna()]
non_marco

In [ ]:
# Selecting students who have a marco learning score through the id being present
marco_students = melted[melted['student_id'].notna()]
marco_students

In [ ]:
# Selecting the rows that have a value for the level
marco_students = marco_students[marco_students['Level'].notna()]
marco_students

In [ ]:
# Creating a dataframe with the performance band level
performance_band = marco_students[marco_students['Standard'].str.contains('Performance Band Level')]
performance_band

In [ ]:
# Replacing the level with the value for performance band
performance_band['Level'] = performance_band['Performance Band Level_y']
performance_band

In [ ]:
# Selecting the rows with the % Correct
percent_correct = marco_students[marco_students['Standard'].str.contains('Percent Correct')]
percent_correct

In [ ]:
# Replacing the level with the value calculated for the % Correct
percent_correct['Level'] = percent_correct['% Correct']
percent_correct

In [ ]:
# Deleting performance bands rows from the Marco dataframe
mask = marco_students['Standard'].str.contains('Performance Band Level')
marco_students = marco_students[~mask]

In [ ]:
marco_students

In [ ]:
# Deleting percent correct rows away from the Marco dataframe
mask = marco_students['Standard'].str.contains('Percent Correct')
marco_students = marco_students[~mask]
marco_students

In [ ]:
# Concatenating to put all Marco dataframes back together with numbers corrected now
marco_students = pd.concat([marco_students, performance_band, percent_correct])
marco_students

In [ ]:
non_marco

In [ ]:
# Concatenating the non-marco students with the marco students
full_students = pd.concat([non_marco, marco_students])
full_students

In [ ]:
# Dropping the extra columns from Marco learning
columns_to_drop = ['student_id', 'Performance Band Level_y', '% Correct']
full_students = full_students.drop(columns=columns_to_drop)

In [ ]:
# Unstacking data to recreate the original Illuminate file with numbers now fixed
unmelted = full_students.groupby(['Student ID', 'Current Site ID', 'Current Grade Level', 'State Student ID',
             'First Name', 'Last Name', 'English Proficiency Code',
            'English Proficiency', 'Student Is Special Ed?', 'Date Taken', 'Standard'])['Level'].max().unstack()
unmelted

In [ ]:
# Changing name of column back to the original
unmelted = unmelted.rename(columns={'Performance Band Level_x': 'Performance Band Level'})
unmelted

In [ ]:
writer = pd.ExcelWriter(overall_final)
final_overall.to_excel(writer, sheet_name='Overall', index=False)
writer.save()              

In [ ]:
writer = pd.ExcelWriter(standards_final)
unmelted.to_excel(writer, sheet_name='Overall', index=True)
writer.save()     